In [ ]:
import openai
import os
import sys
import json
import pandas as pd
import json
import time

from dotenv import load_dotenv
from pathlib import Path

%load_ext autoreload
%autoreload 2
# append parent directory to path to import dataset_utils.py
# sys.path.append(os.path.join(os.path.dirname(os.getcwd()), "functions'))

# colocar "" caso sejo no tecgraf
run_environment = ""
# run_environment = ""

parent_dir = os.path.dirname(os.getcwd())
sys.path.append(os.path.dirname(os.path.dirname(os.getcwd())))
dotenv_path = Path(parent_dir)
load_dotenv(dotenv_path=dotenv_path)

if run_environment == "pt/":
    from functions.pt.chatgpt_utils import get_openai_function_call, get_openai_response_msg
    import functions.pt.gptconfig as gptconfig
    gptconfig.engine  = gptconfig.engine_4
    
else:
    from functions.llama_helper import format_prompt, LlamaHelper
    from functions.dataset_utils import DatasetEvaluator
    from functions.chatgpt_utils import get_openai_function_call, get_openai_response_msg


In [2]:
print(get_openai_response_msg([{"role": "system", "content": "You are a helpful assistant."}]))

{
  "role": "assistant",
  "content": "How can I assist you today?",
  "usage": {
    "prompt_tokens": 13,
    "completion_tokens": 7,
    "total_tokens": 20
  }
}
[-0.010191123001277447, -0.010742533020675182, 0.0006008211639709771, -0.03130415454506874, -0.018774520605802536, 0.03717700392007828, -0.04025958478450775, -0.0129481740295887, 0.008218004368245602, -0.015040875412523746, -0.0025461199693381786, 0.020156366750597954, 0.0037270013708621264, -0.007214836776256561, -0.02519213780760765, 0.0025261894334107637, 0.012569494545459747, -0.009752652607858181, 0.016768183559179306, 0.00037867928040213883, -0.03470562398433685, 0.0075403680093586445, 0.012602712027728558, -0.003142373636364937, -0.023850150406360626, 0.013752036727964878, 0.0068693747743964195, -0.02492639794945717, 0.023903299123048782, -0.005324762314558029, 0.034679047763347626, 0.006205025129020214, 0.006155198905616999, -0.022096268832683563, -0.027955831959843636, -0.01955845206975937, 0.001906683435663581, -0.

## Parâmetros

In [ ]:
model = "gpt-4"
# model = "gpt-3.5-turbo" # "gpt-3.5-turbo-16k" ou "llama2"
dataset_name = "mondial" # "mondial" 
queries_filename = f"results/queries_{dataset_name}_{model}.json"
start_index = 0 # índice da primeira instância do dataset a ser avaliada
end_index = 99 # índice da última instância do dataset avaliada
# índices: use 0 e 99 para o dataset inteiro, use o mesmo número nos dois para só um exemplo


## Setup

### Classe Stage

In [ ]:
class Stage:
    def __init__(self,
                 model="llama2", 
                 system_message="You are a helpful assistant"):
        self.system_message = system_message
        self.messages = [
            {"role": "system", "content": system_message},
            ]
        self.model = model
        self.functions = {}
        self.max_function_depth = 8
        self.function_call = "auto"
        self.token_usage = {
            "completion_tokens": 0,
            "prompt_tokens": 0,
            "total_tokens": 0
        }

    def run(self, msg_content, return_function_arguments=True, model=None, log=False):
        if model is None:
            model = self.model
        response_message = self.get_response({"role": "user", "content": msg_content}, model=model, log=log)

        if not return_function_arguments:
            return response_message
        
        if "function_call" in response_message:
            return json.loads(response_message["function_call"]["arguments"])
        return response_message

    def get_response(self, message, model=None, log=False):
        if model is None:
            model=self.model
        if model == "llama2":
            return
        self.messages.append(message)

        if self.functions:
            response_message = get_openai_function_call(
                messages=self.messages,
                temperature=0,
                max_tokens=1000,
                functions=list(self.functions[func_name]['info'] for func_name in self.functions),
                function_call=self.function_call #,
                #model=model
            )
        else:
            response_message = get_openai_response_msg(
                messages=self.messages,
                temperature=0,
                max_tokens=1000 #,
                #model=model
            )

        if log:
            with open(f"prompt_chain_log{model}.txt", 'a') as f:
                f.write(str(response_message))

        self.messages.append(response_message)

        token_usage = response_message["usage"]
        self.token_usage["completion_tokens"] += token_usage["completion_tokens"]
        self.token_usage["prompt_tokens"] += token_usage["prompt_tokens"]
        self.token_usage["total_tokens"] += token_usage["total_tokens"]

        return response_message

    def add_function(self, func_info, func_callback = None):
        self.functions[func_info["name"]] = {"callback": func_callback, "info": func_info}

    def reset_history(self):
        self.messages = [
            {"role": "system", "content": self.system_message},
        ]
        self.token_usage = {
            "completion_tokens": 0,
            "prompt_tokens": 0,
            "total_tokens": 0
        }

    def print(self, print_system_message=False):
        for message in self.messages:
            if message['role'] == "system" and not print_system_message:
                continue
            if "content" in message:
                print(f"[{message['role']}]: {message['content']}")
            if "function_call" in message:
                print(f"[function call: {message['function_call']['name']}]: {message['function_call']['arguments']}")

    def print_functions(self):
        print(list(self.functions[func_name]['info'] for func_name in self.functions))

In [ ]:
def extract_json(text):
    """
    Find and extract JSON object from strings containing a single JSON object surrounded by other text.
    """
    first_bracket = text.index("{")
    last_bracket = len(text) - text[::-1].index("}")
    json_string = text[first_bracket:last_bracket]
    return json.loads(json_string)

In [ ]:
class LlamaStage():
    def __init__(self, api_url,
                 system_message="Você é um assistente prestativo e educado."):
        self.helper = LlamaHelper(api_url, system_message)

    def run(self, msg_content):
        response = self.helper.generate_response(msg_content)
        return response

### Definição das etapas

#### GPT

In [ ]:
def init_stages_gpt(prompts):
    global stage_1, stage_2, stage_3, stage_4
    with open(prompts[0], 'r', encoding="utf8") as f:
        system_message_1 = f.read()

        stage_1 = Stage(system_message=system_message_1)

        stage_1.add_function({
                            "name": "return_entities",
                                "description": "Returns the list of entities to be used in the query to the system",
                                "parameters": {
                                    "type": "object",
                                    "properties": {
                                        "reasoning": {
                                            "type": "string",
                                            "description": "Reasoning behind choice of entities"
                                        },
                                        "entities": {
                                            "type": "array",
                                            "description": "List of entities",
                                            "items": {
                                                "type": "string",
                                            }
                                        },
                                    },
                                    "required": ["reasoning", "entities"],
                                },
                            })
        stage_1.function_call = {"name": "return_entities"}
    
    with open(prompts[1], 'r', encoding="utf8") as f:
        system_message_2 = f.read()

        stage_2 = Stage(system_message=system_message_2)

        stage_2.add_function({
                            "name": "return_properties",
                                "description": "Returns the list of properties to be used in the query to the system",
                                "parameters": {
                                    "type": "object",
                                    "properties": {
                                        "reasoning": {
                                            "type": "string",
                                            "description": "Reasoning behind choice of properties"
                                        },
                                        "entity_properties": {
                                            "type": "array",
                                            "description": "List of objects containing entity names and their properties to be included in the query to the system",
                                            "items": {
                                                "type": "object",
                                                "properties": {
                                                    "entity": {
                                                        "type": "string",
                                                        "description": "Entity name"
                                                    },
                                                    "properties": {
                                                        "type": "array",
                                                        "description": "List of included properties corresponding to the entity",
                                                        "items": {
                                                            "type": "string"
                                                        }
                                                    }
                                                }
                                            }
                                        },
                                    },
                                    "required": ["reasoning", "entity_properties"],
                                },
                            })

        stage_2.function_call = {"name": "return_properties"}

    with open(prompts[2], 'r', encoding="utf8") as f:
        system_message_3 = f.read()

        stage_3 = Stage(system_message=system_message_3)

        stage_3.add_function({
                            "name": "return_queries",
                                "description": "Returns the list of possible keyword queries to the user",
                                "parameters": {
                                    "type": "object",
                                    "properties": {
                                        "reasoning": {
                                            "type": "string",
                                            "description": "Reasoning behind query strings"
                                        },
                                        "queries": {
                                            "type": "array",
                                            "description": "List of query strings",
                                            "items": {
                                                "type": "string"
                                            }
                                        },
                                    },
                                    "required": ["reasoning", "queries"],
                                },
                            })

        stage_3.function_call = {"name": "return_queries"}

    with open(prompts[3], 'r', encoding="utf8") as f:
        system_message_4 = f.read()

        stage_4 = Stage(system_message=system_message_4)

        stage_4.add_function({
                            "name": "return_query",
                                "description": "Returns the final query string to the system",
                                "parameters": {
                                    "type": "object",
                                    "properties": {
                                        "reasoning": {
                                            "type": "string",
                                            "description": "Reasoning behind choice of best query"
                                        },
                                        "query_string": {
                                            "type": "string",
                                            "description": "Final query string",
                                        },
                                    },
                                    "required": ["reasoning", "query_string"],
                                },
                            })

        stage_4.function_call = {"name": "return_query"}

#### Llama 2

In [ ]:
def init_stages_llama(prompts):
    global stage_1, stage_2, stage_3, stage_4

    with open("llama.json", 'r') as f:
        api_url = json.load(f)["API_URL"]

    with open(prompts[0], 'r', encoding="utf8") as f:
        system_message_1 = f.read()

        stage_1 = LlamaStage(api_url, system_message=system_message_1)
    
    with open(prompts[1], 'r', encoding="utf8") as f:
        system_message_2 = f.read()

        stage_2 = LlamaStage(api_url, system_message=system_message_2)

    with open(prompts[2], 'r', encoding="utf8") as f:
        system_message_3 = f.read()

        stage_3 = LlamaStage(api_url, system_message=system_message_3)

    with open(prompts[3], 'r', encoding="utf8") as f:
        system_message_4 = f.read()

        stage_4 = LlamaStage(api_url, system_message=system_message_4)

### Funções auxiliares

In [ ]:
def format_properties(properties_json):
    s = "Entity - Property: Description\n"
    for prop in properties_json:
        entity_name = prop['entity']
        prop_name = prop['property']
        if ' ' in prop_name:
            prop_name = f'"{prop_name}"'
        if ' ' in entity_name:
            entity_name = f'"{entity_name}"'
        if "description" in prop:
            s += f"{entity_name} - {prop_name}: {prop['description']}\n"
        else:
            s += f"{entity_name} - {prop_name}\n"
    return s

In [ ]:
def get_entities_info(entities, include_description=True):
    df = pd.DataFrame()
    for entity in entities:
        df = pd.concat([df, db_info[db_info["entity"] == entity]])
    if not include_description:
        df = df.drop(columns="description")
    return format_properties(json.loads(df.to_json(orient='records', force_ascii=False)))
# json.loads(db_info[db_info["entity"] == "País"].to_json(orient='records'))

In [ ]:
def get_properties_info(entity_properties, include_description=True):
    properties = []
    for property_list in entity_properties:
        entity = property_list["entity"]
        for prop in property_list["properties"]:
            properties.append((entity, prop))
    df = pd.DataFrame()
    for prop in properties:
        df = pd.concat([df, db_info[(db_info["entity"] == prop[0]) & (db_info["property"] == prop[1])]])
    if not include_description:
        df = df.drop(columns="description")
    return format_properties(json.loads(df.to_json(orient='records', force_ascii=False)))

### Definição do "pipeline"

#### GPT

In [ ]:
def question_to_query_gpt(user_query, verbose=False, model=None, log=True):
    token_usage = {
        "completion_tokens": 0,
        "prompt_tokens": 0,
        "total_tokens": 0
    }

    stage_1.reset_history()
    if log:
        with open(f"prompt_chain_log{model}.txt", 'a') as f:
            f.write(f"\nStage 1 - Query: {user_query}\n")
    stage_1_response = stage_1.run(user_query, model=model, log=True)

    if verbose:
        print("\n### Stage 1")
        stage_1.print()

    token_usage["completion_tokens"] += stage_1.token_usage["completion_tokens"]
    token_usage["prompt_tokens"] += stage_1.token_usage["prompt_tokens"]
    token_usage["total_tokens"] += stage_1.token_usage["total_tokens"]

    entities = stage_1_response['entities']
    entity_info = get_entities_info(entities)

    ### Stage 2

    stage_2_input = \
    f"""
####
Busca do usuário: {user_query}
Entidades a serem utilizadas: {entities}
Descrição de suas propriedades:
{entity_info}
####
"""

    if log:
        with open(f"prompt_chain_log{model}.txt", 'a') as f:
            f.write(f"\nStage 2 - Query: {user_query}\n")
    try:
        stage_2.reset_history()
        stage_2_response = stage_2.run(stage_2_input, model=model, log=True)
    except:
        stage_2.reset_history()
        if verbose:
            print("Error, trying shorter prompt")
        if log:
            with open(f"prompt_chain_log{model}.txt", 'a') as f:
                f.write("\nError, trying shorter prompt\n")
        short_entity_info = get_entities_info(entities, include_description=False)
        stage_2_input = \
    f"""
####
Busca do usuário: {user_query}
Entidades a serem utilizadas: {entities}
Descrição de suas propriedades:
{short_entity_info}
####
"""
        if verbose:
            print(stage_2_input)
        stage_2_response = stage_2.run(stage_2_input, model=model, log=True)

    if verbose:
        print("\n### Stage 2")
        stage_2.print()

    token_usage["completion_tokens"] += stage_2.token_usage["completion_tokens"]
    token_usage["prompt_tokens"] += stage_2.token_usage["prompt_tokens"]
    token_usage["total_tokens"] += stage_2.token_usage["total_tokens"]

    entity_properties = stage_2_response['entity_properties']

    ### Stage 3

    stage_3_input = \
    f"""
####
Busca do usuário: {user_query}
Entidades a serem utilizadas: {entities}
Propriedades a serem utilizadas: {get_properties_info(entity_properties)}
####
"""

    if log:
        with open(f"prompt_chain_log{model}.txt", 'a') as f:
            f.write(f"\nStage 3 - Query: {user_query}\n")
    try:
        stage_3.reset_history()
        stage_3_response = stage_3.run(stage_3_input, model=model, log=True)
    except:
        stage_3.reset_history()
        if verbose:
            print("Error, trying shorter prompt")
        if log:
            with open(f"prompt_chain_log{model}.txt", 'a') as f:
                f.write("\nError, trying shorter prompt\n")
        stage_3_input = \
    f"""
####
Busca do usuário: {user_query}
Entidades a serem utilizadas: {entities}
Propriedades a serem utilizadas: {get_properties_info(entity_properties, include_desctiption=False)}
####
"""
        stage_3_response = stage_3.run(stage_3_input, model=model, log=True)

    if verbose:
        print("\n### Stage 3")
        stage_3.print()

    token_usage["completion_tokens"] += stage_3.token_usage["completion_tokens"]
    token_usage["prompt_tokens"] += stage_3.token_usage["prompt_tokens"]
    token_usage["total_tokens"] += stage_3.token_usage["total_tokens"]

    queries = stage_3_response['queries']

    ### Stage 4

    stage_4_input = \
f"""
####
Busca do usuário:
{user_query}
Consultas possíveis:
{queries}
####
"""

    stage_4.reset_history()
    if log:
        with open(f"prompt_chain_log{model}.txt", 'a') as f:
            f.write(f"\nStage 4 - Query: {user_query}\n")
    stage_4_response = stage_4.run(stage_4_input, model=model, log=True)

    if verbose:
        print("\n### Stage 4")
        stage_4.print()

    query_string = stage_4_response['query_string']

    if verbose:
        print(f"Query: {query_string}")
    
    token_usage["completion_tokens"] += stage_4.token_usage["completion_tokens"]
    token_usage["prompt_tokens"] += stage_4.token_usage["prompt_tokens"]
    token_usage["total_tokens"] += stage_4.token_usage["total_tokens"]

    return query_string, token_usage

#### Llama 2

In [ ]:
def question_to_query_llama(user_query, verbose=False, model=None):
    stage_1_input = json.dumps({
        "user_input": user_query
    })
    stage_1_response = stage_1.run(stage_1_input)
    stage_1_json = extract_json(stage_1_response)

    if verbose:
        print("\n\n### Stage 1")
        print(f"Input:\n{stage_1_input}\nOutput:\n{stage_1_json}")

    entities = stage_1_json['entities']
    entity_info = get_entities_info(entities)

    ### Stage 2

    stage_2_input = json.dumps({
        "user_input": user_query,
        "entities": entities,
        "properties": entity_info
    })

    if verbose:
        print(f"\nEntities extracted from 1:\n{entities}")
        print(f"Stage 2 input:\n{stage_2_input}")

    stage_2_response = stage_2.run(stage_2_input)
    if verbose:
        print(f"Stage 2 response:\n{stage_2_response}")
        
    stage_2_json = extract_json(stage_2_response)

    if verbose:
        print("\n\n### Stage 2")
        print(f"Input:\n{stage_2_input}\nOutput:\n{stage_2_json}")

    entity_properties = stage_2_json['properties']

    ### Stage 3

    stage_3_input = json.dumps({
        "user_input": user_query,
        "entities": entities,
        "properties": entity_properties
    })

    if verbose:
        print(f"Properties extracted from 2:\n{entity_properties}")
        print(f"Stage 3 input:\n{stage_3_input}")

    stage_3_response = stage_3.run(stage_3_input)
    if verbose:
        print(f"Stage 3 response:\n{stage_3_response}")

    stage_3_json = extract_json(stage_3_response)

    if verbose:
        print("\n### Stage 3")
        print(f"Input:\n{stage_3_input}\nOutput:\n{stage_3_json}")

    queries = stage_3_json['queries']

    ### Stage 4

    stage_4_input = json.dumps({
        "user_input": user_query,
        "queries": queries
    })

    if verbose:
        print(f"Queries extracted from 3:\n{queries}")
        print(f"Stage 4 input:\n{stage_4_input}")

    stage_4_response = stage_3.run(stage_4_input)
    if verbose:
        print(f"Stage 4 response:\n{stage_4_response}")

    stage_4_json = extract_json(stage_4_response)

    if verbose:
        print("\n### Stage 4")
        print(f"Input:\n{stage_4_input}\nOutput:\n{stage_4_json}")

    final_query = stage_4_json['query']
    
    return final_query

In [ ]:
def question_to_query(user_query, prompts, verbose=False, model="llama2"):
    if "gpt" in model:
        init_stages_gpt(prompts)
        return question_to_query_gpt(user_query, verbose=verbose, model=model)
    elif model == "llama2":
        init_stages_llama(prompts)
        return question_to_query_llama(user_query, verbose=verbose), None
    return "", None

## Avaliação 

### Geração de consultas

In [ ]:
# dataset_name = "mondial"
# model = "gpt-3.5-turbo"
# model = "llama2"


if dataset_name == "mondial":
    dataset_file_path = "../../datasets/mondial/mondial_dataset.json"
    dataset_tables_path = "../../datasets/mondial/result_tables/"
    db_connection_file = "../../datasets/mondial_db_connection.json"

    prompt_dir = "prompts_mondial/"

    db_info = pd.read_json('db_info_mondial.json')

if "gpt" in model:
    prompts = [
        f"{prompt_dir}prompt1.txt",
        f"{prompt_dir}prompt2.txt",
        f"{prompt_dir}prompt3.txt",
        f"{prompt_dir}prompt4.txt"
    ]
elif model == "llama2":
    prompts = [
        f"{prompt_dir}prompt1_llama.txt",
        f"{prompt_dir}prompt2_llama.txt",
        f"{prompt_dir}prompt3_llama.txt",
        f"{prompt_dir}prompt4_llama.txt"
    ]

with open(dataset_file_path, 'r', encoding="utf8") as f:
    dataset = json.load(f)["dataset"]

In [ ]:
def generate_queries(start_index, end_index, id_list=None, model=None, verbose=False, save_filename=None, override=True, delay=0):
    if save_filename:
        try:
            with open(save_filename, 'r', encoding="utf8") as f:
                queries = json.load(f)
        except:
            queries = {}

        if save_filename[-5:] == ".json":
            tokens_filename = save_filename[:-5] + "_tokens" + ".json"
        else:
            tokens_filename = save_filename + "_tokens.json"
        
        try:
            with open(tokens_filename, 'r', encoding="utf8") as f:
                queries_tokens = json.load(f)
        except:
            queries_tokens = {}
    else:
        queries = {}
    for i in range(start_index, end_index + 1):
        dataset_instance = dataset[i]
        if not override and dataset_instance["id"] in queries.keys():
            continue
        if id_list is not None and dataset_instance["id"] not in id_list:
            continue
        question = dataset_instance["question"]
        time_elapsed = 0
        try:
            start = time.time()
            query, token_usage = question_to_query(question, prompts, verbose=verbose, model=model)
            end = time.time()
            time_elapsed = end - start
        except Exception as exception:
            print("Error generating query.")
            print(f"ID: {dataset_instance['id']}\nQuery: {question}")
            print(f"Exception: {str(exception)}")
            query = ""
            token_usage = {
                "completion_tokens": 0,
                "prompt_tokens": 0,
                "total_tokens": 0,
            }
        queries[dataset_instance["id"]] = query
        queries_tokens[dataset_instance["id"]] = token_usage
        queries_tokens[dataset_instance["id"]]["time_elapsed"] = time_elapsed
        if save_filename:
            with open(save_filename, 'w', encoding="utf8") as f:
                json.dump(queries, f, indent=4, ensure_ascii=False)
            with open(tokens_filename, 'w', encoding="utf8") as f:
                json.dump(queries_tokens, f, indent=4, ensure_ascii=False)
        time.sleep(delay)
            
    return queries

In [ ]:
ids = ["16", "21", "23", "36", "37", "45", "47", "50", "52", "53", "78", "79", "80", "81", "83", "84", "85", "86"]

In [ ]:
# queries_filename="queries_mondial_llama2_13b.json"
queries_filename = "queries_mondial_gpt4_100_2.json"
queries = generate_queries(start_index, end_index, id_list=ids, model=model, verbose=True, save_filename=queries_filename, override=True, delay=8)

### Avaliação dos resultados

In [ ]:
# eval_filename = queries_filename
eval_filename = "queries_mondial_gpt35_100.json"
eval_dataset_name = "mondial"

In [ ]:
if eval_dataset_name == "mondial":
    eval_dataset_file_path = "../../datasets/mondial/mondial_dataset.json"
    eval_dataset_tables_path = "../../datasets/mondial/result_tables/"
    eval_db_connection_file = "../../datasets/mondial_db_connection.json"

    eval_db_info = pd.read_json('db_info_mondial.json')



with open(eval_dataset_file_path, 'r', encoding="utf8") as f:
    eval_dataset = json.load(f)["dataset"]

In [ ]:
evaluator = DatasetEvaluator(dataset_file_path=eval_dataset_file_path,
                             dataset_tables_path=eval_dataset_tables_path,
                             db_connection_file=eval_db_connection_file,
                             dataset_name=eval_dataset_name)

In [ ]:
with open(eval_filename, 'r', encoding="utf8") as f:
    queries_raw = json.load(f)
queries = []
for instance_id, query in queries_raw.items():
    queries.append({
        "id": instance_id,
        # "query_string": query
        "query_string": query.replace("'", '"')
    })

In [ ]:
queries

In [ ]:
result = evaluator.evaluate_query_batch(queries, verbose=True, delay=1)

In [ ]:
result

In [ ]:
result_totals = {
    "correct": {
        "all": 0,
        "simple": 0,
        "medium": 0,
        "complex": 0
    },
    "total": {
        "all": 0,
        "simple": 0,
        "medium": 0,
        "complex": 0
    }
}
for r in result:
    complexity = r["complexity"]
    result_totals["total"]["all"] += 1
    result_totals["total"][complexity] += 1
    if r["correct"]:
        result_totals["correct"]["all"] += 1
        result_totals["correct"][complexity] += 1

def calculate_percentage(correct, total):
    return (correct / total) * 100 if total != 0 else 0

print("Results:")
for complexity in ["all", "simple", "medium", "complex"]:
    correct = result_totals["correct"][complexity]
    total = result_totals["total"][complexity]
    percentage = calculate_percentage(correct, total)
    print(f"{complexity.capitalize():7s} - {correct:02d}/{total:02d} ({percentage:6.2f}%)")

In [ ]:
with open("results_100/results_mondial_gpt35_100.json", 'w') as f:
    json.dump(result, f, indent=4)

In [ ]:
metadata_filename = "queries_mondial_gpt35_100_tokens.json"
with open(metadata_filename, 'r') as f:
    metadata = json.load(f)

total_time = 0
for id in metadata.keys():
    total_time += metadata[id]["time_elapsed"]

time_per_query = total_time / len(metadata.keys())

time_per_query